In [2]:
import logging
from itertools import combinations

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
from icecream import ic

In [3]:
class Problem:
    _graph: nx.Graph
    _alpha: float
    _beta: float

    def __init__(
        self,
        num_cities: int,
        *,
        alpha: float = 1.0,
        beta: float = 1.0,
        density: float = 0.5,
        seed: int = 42,
    ):
        rng = np.random.default_rng(seed)
        self._alpha = alpha
        self._beta = beta
        cities = rng.random(size=(num_cities, 2))
        cities[0, 0] = cities[0, 1] = 0.5

        self._graph = nx.Graph()
        self._graph.add_node(0, pos=(cities[0, 0], cities[0, 1]), gold=0)
        for c in range(1, num_cities):
            self._graph.add_node(c, pos=(cities[c, 0], cities[c, 1]), gold=(1 + 999 * rng.random()))

        tmp = cities[:, np.newaxis, :] - cities[np.newaxis, :, :]
        d = np.sqrt(np.sum(np.square(tmp), axis=-1))
        for c1, c2 in combinations(range(num_cities), 2):
            if rng.random() < density or c2 == c1 + 1:
                self._graph.add_edge(c1, c2, dist=d[c1, c2])

        assert nx.is_connected(self._graph)

    @property
    def graph(self) -> nx.Graph:
        return nx.Graph(self._graph)

    @property
    def dist_dict(self):
        return {(u, v): data['dist'] for u, v, data in self._graph.edges(data=True)}

    @property
    def gold_dict(self):
        return {n: data['gold'] for n, data in self._graph.nodes(data=True)}

    def cost(self, path, weight):
        dist = nx.path_weight(self._graph, path, weight='dist')
        return dist + (self._alpha * dist * weight) ** self._beta

    def baseline(self):
        total_cost = 0
        for dest, path in nx.single_source_dijkstra_path(
            self._graph, source=0, weight='dist'
        ).items():
            cost = 0
            for c1, c2 in zip(path, path[1:]):
                cost += self.cost([c1, c2], 0)
                cost += self.cost([c1, c2], self._graph.nodes[dest]['gold'])
            logging.debug(
                f"dummy_solution: go to {dest} ({' > '.join(str(n) for n in path)} ({cost})"
            )
            total_cost += cost
        return total_cost

    def plot(self):
        plt.figure(figsize=(10, 10))
        pos = nx.get_node_attributes(self._graph, 'pos')
        size = [100] + [self._graph.nodes[n]['gold'] for n in range(1, len(self._graph))]
        color = ['red'] + ['lightblue'] * (len(self._graph) - 1)
        return nx.draw(self._graph, pos, with_labels=True, node_color=color, node_size=size)

In [4]:
logging.getLogger().setLevel(logging.WARNING)

ic(Problem(100, density=0.2, alpha=1, beta=1).baseline())
ic(Problem(100, density=0.2, alpha=2, beta=1).baseline())
ic(Problem(100, density=0.2, alpha=1, beta=2).baseline())
ic(Problem(100, density=1, alpha=1, beta=1).baseline())
ic(Problem(100, density=1, alpha=2, beta=1).baseline())
ic(Problem(100, density=1, alpha=1, beta=2).baseline())
ic(Problem(1_000, density=0.2, alpha=1, beta=1).baseline())
ic(Problem(1_000, density=0.2, alpha=2, beta=1).baseline())
ic(Problem(1_000, density=0.2, alpha=1, beta=2).baseline())
ic(Problem(1_000, density=1, alpha=1, beta=1).baseline())
ic(Problem(1_000, density=1, alpha=2, beta=1).baseline())
ic(Problem(1_000, density=1, alpha=1, beta=2).baseline())

None

ic| Problem(100, density=0.2, alpha=1, beta=1).baseline(): np.float64(25266.40561851072)
ic| Problem(100, density=0.2, alpha=2, beta=1).baseline(): np.float64(50425.30961817918)
ic| Problem(100, density=0.2, alpha=1, beta=2).baseline(): np.float64(5334401.927002504)
ic| Problem(100, density=1, alpha=1, beta=1).baseline(): np.float64(18266.18579582672)
ic| Problem(100, density=1, alpha=2, beta=1).baseline(): np.float64(36457.918462372065)
ic| Problem(100, density=1, alpha=1, beta=2).baseline(): np.float64(5404978.08899582)
ic| Problem(1_000, density=0.2, alpha=1, beta=1).baseline(): np.float64(195402.95810394012)
ic| Problem(1_000, density=0.2, alpha=2, beta=1).baseline(): np.float64(390028.72126288974)
ic| Problem(1_000, density=0.2, alpha=1, beta=2).baseline(): np.float64(37545927.70213464)
ic| Problem(1_000, density=1, alpha=1, beta=1).baseline(): np.float64(192936.23377726765)
ic| Problem(1_000, density=1, alpha=2, beta=1).baseline(): np.float64(385105.64149576554)
ic| Problem(1_000

In [5]:
def calculate_full_path_cost_final(problem_instance, path, trip_counts=None):
    
    total_cost = 0.0
    current_weight = 0.0
    infeasible_nodes = []
    
    dist_dict = problem_instance.dist_dict
    gold_dict = problem_instance.gold_dict
    
    total_trips_in_path = 0
    for i in range(len(path) - 1):
        if path[i+1][0] == 0:
            total_trips_in_path += 1
            
    # Se trip_counts non è fornito, assume 1 per ogni circuito
    if trip_counts is None:
        trip_counts = [1] * total_trips_in_path
    
    # Indice per scorrere trip_counts
    trip_index = 0
    # Imposta il conteggio per il primo circuito
    current_trip_count = trip_counts[trip_index] if trip_index < len(trip_counts) else 1
    
    for i in range(len(path) - 1):
        u = path[i][0]
        v = path[i + 1][0]
        u_flag = path[i][1]
        
        # 1. SALVIAMO IL MOLTIPLICATORE CORRENTE
        # Dobbiamo usare il conteggio del viaggio corrente per calcolare il costo di QUESTO arco.
        # Se v == 0, l'indice globale cambierà per il prossimo arco, ma per questo arco vale ancora il vecchio.
        this_segment_trips = current_trip_count

        # 2. GESTIONE CARICO (Load Splitting)
        # Se il nodo è attivo, raccogliamo 1/N dell'oro disponibile
        if u != 0 and u_flag == True:
            gold_at_u = gold_dict.get(u, 0)
            current_weight += gold_at_u / this_segment_trips
        
        # 3. CALCOLO COSTO UNITARIO (Del singolo viaggio)
        dist_uv = dist_dict.get((min(u, v), max(u, v)))
        if dist_uv is None:
            dist_uv = 1.0 # Penalità default
            
        # Formula: dist + (alpha * dist * weight)^beta
        # Qui current_weight è ridotto (weight_totale / N)
        cost_segment_unit = dist_uv + (problem_instance._alpha * dist_uv * current_weight) ** problem_instance._beta
        
        # 4. AGGIUNTA AL TOTALE MOLTIPLICATO PER N
        # Siccome facciamo "this_segment_trips" viaggi uguali su questo arco:
        total_cost += cost_segment_unit * this_segment_trips

        # 5. GESTIONE FINE CIRCUITO (Preparazione per iterazione successiva)
        if v == 0:
            # Siamo tornati al deposito: il peso si resetta per il prossimo giro (che partirà da 0 vuoto)
            current_weight = 0.0
            
            # Passiamo al prossimo circuito nella lista trip_counts
            trip_index += 1
            if trip_index < len(trip_counts):
                current_trip_count = trip_counts[trip_index]
            else:
                current_trip_count = 1 # Fallback
                
        # Reset peso se ripartiamo da 0 (ridondante ma sicuro)
        if u == 0:
            current_weight = 0.0 # Assicura che si parta vuoti
    
    return (len(infeasible_nodes), total_cost)


def count_trips(path):
    """
    Conta il numero di circuiti (viaggio da 0 a 0) in un percorso.
    """
    count = 0
    for node, _ in path:
        if node == 0:
            count += 1
    return count


def get_trip_boundaries(path):
    """
    Restituisce una lista di tuple (start_idx, end_idx) che delimita ogni circuito.
    Esempio: Se il path ha 2 circuiti, restituisce [(0, idx1), (idx1, idx2)]
    """
    boundaries = []
    trip_start = 0
    
    for i, (node, _) in enumerate(path):
        if node == 0 and i > 0:
            boundaries.append((trip_start, i))
            trip_start = i
    
    # Aggiungi l'ultimo circuito (che termina all'ultimo elemento)
    if trip_start < len(path):
        boundaries.append((trip_start, len(path) - 1))
    
    return boundaries

In [6]:
def neighborhood_greedy_strategy_dijistra(problem_instance):

    graph = problem_instance.graph
    node_to_path =dict(nx.single_source_dijkstra_path(graph, source=0, weight='dist'))

    # Crea path_list: lista dei percorsi di ogni nodo ordinati per nodo ID
    path_list = [node_to_path[node] for node in sorted(node_to_path.keys())]
    

    return path_list


In [7]:
def choice_a_path(path_list,seed=42):
    full_path=[]
    rng=random.Random(seed)
    ungolden_nodes=[i for i in range(1, len(path_list))]
    while ungolden_nodes:
        
        node=rng.choice(ungolden_nodes)
        
        path_for_node=list(path_list[node].copy())
        for p in path_for_node[:-1]:
            full_path.append((p, False))
        path_for_node.reverse()
        

        for  p in (path_for_node[:-1]):

            if p in ungolden_nodes:
                full_path.append((p, True))
                ungolden_nodes.remove(p)
                
            else:
                full_path.append((p,False))
            
    full_path.append((0,False))
    return(full_path)


In [8]:
def precompute_neighbor_distances(problem_instance):
    graph = problem_instance.graph
    neighbor_dist_cache = {}
    
    for node in graph.nodes():
        neighbors_dict = {}
        node_coords = graph.nodes[node]['pos']
        
        for neighbor in graph.neighbors(node):
            if neighbor != 0:
                n_coords = graph.nodes[neighbor]['pos']
                dist = ((n_coords[0] - node_coords[0])**2 + 
                        (n_coords[1] - node_coords[1])**2) ** 0.5
                neighbors_dict[neighbor] = dist
        
        neighbor_dist_cache[node] = neighbors_dict
    
    return neighbor_dist_cache

In [9]:
def find_node_with_flag_true(path, wanted_node):

    for i in range(len(path)):
        if path[i][1] and path[i][0] == wanted_node:
            return i
    return -1

In [10]:
def founding_start_and_end_index(path, start_index):
    end_index=start_index+1
    while path[start_index][0]!=0:
        start_index-=1
    while path[end_index][0]!=0:
        end_index+=1

   
    return start_index, end_index

In [11]:
def find_True_flags(path_segment):
    gold_elements=[]
    for node, flag in path_segment:
        if flag:
            gold_elements.append(node)

    return gold_elements

In [12]:
def remove_more_gold_nodes(parent_path, gold_nodes, start_index_2, end_index_2):
    new_path=parent_path.copy()
    index=find_node_with_flag_true(new_path, gold_nodes)
    
    start_index, end_index = founding_start_and_end_index(new_path, index)
    list_of_gold_nodes_in_segment = find_True_flags(new_path[start_index:end_index])
    if len(list_of_gold_nodes_in_segment)==1:
        new_path=new_path[:start_index]+new_path[end_index:]
        delta=end_index-start_index
        start_index_2=start_index_2-delta if start_index_2 > start_index else start_index_2
        end_index_2=end_index_2-delta if end_index_2 > end_index else end_index_2
        return new_path, start_index_2, end_index_2
    new_path[index]=(new_path[index][0], False)

    return new_path, start_index_2, end_index_2

In [13]:

def find_path_removed(parent_path, gold_nodes):
    new_path=parent_path.copy()
    index=find_node_with_flag_true(new_path, gold_nodes)
    
    start_index, end_index = founding_start_and_end_index(new_path, index)
    list_of_gold_nodes_in_segment = find_True_flags(new_path[start_index:end_index])
    old_insert=new_path[start_index:end_index+1]
    if len(list_of_gold_nodes_in_segment)==1:
        new_path=new_path[:start_index]+new_path[end_index:]
        new_insert=[]
        return new_path, new_insert, old_insert
    new_path[index]=(new_path[index][0], False)
    new_insert=new_path[start_index:end_index+1]


    return new_path, new_insert, old_insert

In [14]:
def insert_more_gold_nodes(gold_nodes, path_list):
    full_path=[]
    seed=random.randint(0,1000)
    rng=random.Random(seed)
    ungolden_nodes=gold_nodes.copy()
    while ungolden_nodes:
        
        node=rng.choice(ungolden_nodes)
        
        path_for_node=list(path_list[node].copy())
        for p in path_for_node[:-1]:
            full_path.append((p, False))
        path_for_node.reverse()
        

        for  p in (path_for_node[:-1]):

            if p in ungolden_nodes:
                full_path.append((p, True))
                ungolden_nodes.remove(p)
                
            else:
                full_path.append((p,False))
            
    full_path.append((0,False))

    return full_path

In [15]:
def crossover_zero_paths(parent1, parent2, possible_paths):
    """Esegue croindssover tra due percorsi, restituendo un nuovo percorso figlio."""
    # Trova un punto di crossover casuale
    start_index_1 = random.randint(1, len(parent1) - 2)
    gold_element=0
    index_gold_element_1=-1
    index_gold_element_2=-1
    new_path=[]
    possible_remove=[]
    start_index_1,end_index_1=founding_start_and_end_index(parent1, start_index_1)

    for i in range(start_index_1,end_index_1):
        if parent1[i][1]==True:
            gold_element=parent1[i][0]
            index_gold_element_1=i
            break

    list_gold_1=find_True_flags(parent1[index_gold_element_1+1:end_index_1])
 
    index_gold_element_2=find_node_with_flag_true(parent2, gold_element)
    


    try:
        start_index_2, end_index_2=founding_start_and_end_index(parent2, index_gold_element_2)
    except IndexError:
        raise ValueError("Path ha perso un elemento gold")
    

    list_gold_2=find_True_flags(parent2[start_index_2:end_index_2])

    if len(list_gold_1)>0:
        for node in list_gold_1:
            if node not in list_gold_2:
                parent2, start_index_2, end_index_2=remove_more_gold_nodes(parent2, node, start_index_2, end_index_2)
                

  
    
    list_gold_2.remove(gold_element)
    list_2_not_in_1=[node for node in list_gold_2 if node not in list_gold_1]

   
    if len(list_2_not_in_1)>0:
        new_path=insert_more_gold_nodes(list_2_not_in_1, possible_paths)
    new_individual=parent2[:start_index_2]+ parent1[start_index_1:end_index_1]+ parent2[end_index_2:]+ new_path[1:]



    
    



    return new_individual

In [16]:
def mutation_neighbor_of_next(path, problem_instance, path_list, percentual=0.5):

    new_path = path.copy()
    graph = problem_instance.graph
    insertion_flag=False
    replaced_flag=False
    node_replaced=0


    random_index = random.randint(1, len(new_path) - 2)
    #trovo il settore che sto andando a modificare
    start_index,end_index=founding_start_and_end_index(new_path, random_index)
    for i in range(start_index,end_index):
        if path[i][1]==True:
            first_true_idx=i
            break
    
    
    #salvo il settore per un confronto parziale
    starting_gene=path[start_index:end_index+1]

    if first_true_idx+1 == end_index:
        insertion_flag=True
        idx_to_mutate=first_true_idx
    else:
        candidates_indices = list(range(first_true_idx + 1, end_index))
        idx_to_mutate = random.choice(candidates_indices)
        
        if new_path[idx_to_mutate][1]:
            replaced_flag=True
            node_replaced=new_path[idx_to_mutate][0]
      
    if random.random()< percentual:
        insertion_flag=True        
    
    next_node = new_path[idx_to_mutate + 1][0]

    #posso implementarlo passando tutti i neighboor insieme    
    neighbors = list(graph.neighbors(next_node))
    if 0 in neighbors:
        neighbors.remove(0)
    if new_path[idx_to_mutate][0] in neighbors:
        neighbors.remove(new_path[idx_to_mutate][0])
    if neighbors==[]:
        return path, (0, 0, [])
    new_node = random.choice(neighbors)

    




    


    if insertion_flag:
        x=nx.shortest_path(graph, source=new_path[idx_to_mutate][0], target=new_node, weight='dist')
        link_path=[]
        for i in range(1, len(x)-1):
            link_path.append((x[i], False))

        new_path=new_path[:idx_to_mutate + 1]+link_path+[(new_node, True)]+new_path[idx_to_mutate + 1:]
    else:
        x=nx.shortest_path(graph, source=new_path[idx_to_mutate-1][0], target=new_node, weight='dist')
        link_path=[]
        for i in range(1, len(x)-1):
            link_path.append((x[i], False))
        new_path=new_path[:idx_to_mutate]+link_path+[(new_node, True)]+new_path[idx_to_mutate + 1:]

    # Cerco dove il nodo era attivo e lo disattivo
    new_path,_,_=remove_more_gold_nodes(new_path,new_node,0,0)



    #se il nodo che ho tolto era attivo lo inserisco nuovamente come nodo da qualche parte
    path_to_append=[]
    if replaced_flag:
        path_to_append=insert_more_gold_nodes([node_replaced], path_list)


        new_path=new_path+path_to_append[1:]


    new_start_index,new_end_index=founding_start_and_end_index(new_path, random_index)

    # FIX: Estrai il segmento da new_path (non da path) per avere il segmento mutato
    new_gene=new_path[new_start_index:new_end_index+1]

    # Calcola correttamente la delta estraendo gli elementi dalla tupla
    new_gene_infeas, new_gene_cost = calculate_full_path_cost_final(problem_instance, new_gene)
    starting_gene_infeas, starting_gene_cost = calculate_full_path_cost_final(problem_instance, starting_gene)
    
    # Delta è una tupla (cambio_infeasibility, cambio_costo, infeasible_nodes)
    delta = (new_gene_infeas - starting_gene_infeas, new_gene_cost - starting_gene_cost)

    return new_path, delta

In [17]:
import networkx as nx
import random
import heapq

def mutation_neighbor_of_next_insertion_only(path, problem_instance, path_list, neighbor_distance_cache=None, graph=None):
    """
    Inserisce un nodo in un segmento con selezione intelligente basata su distanza.
    
    OTTIMIZZAZIONI:
    1. Usa Set O(1) per duplicati invece di loop O(n)
    2. Partial sorting (heapq) invece di full sort
    3. Weighted selection basata su distanza
    4. Cache pre-calcolato per distanze (se fornito)
    5. Adaptive retry basato su occupancy
    6. Graph cachato per evitare copie!
    """
    if graph is None:
        graph = problem_instance.graph
    
    # Build active nodes set
    active_nodes = set(n for n, flag in path if flag)
    active_count = len(active_nodes)
    total_nodes = len(graph)
    occupancy = active_count / total_nodes
    
    # Adaptive retries
    max_retries = max(1, 4 - int(occupancy * 3))
    
    for attempt in range(max_retries):
        # Selezione segmento
        random_index = random.randint(1, len(path) - 2)
        s_ins, e_ins = founding_start_and_end_index(path, random_index)
        
        # Primo nodo attivo
        first_true_idx = -1
        for i in range(s_ins, e_ins):
            if path[i][1]:
                first_true_idx = i
                break
        
        if first_true_idx == -1: 
            continue 

        # Sceglie dove inserire
        if first_true_idx + 1 >= e_ins:
            idx_to_mutate = first_true_idx
        else:
            candidates = list(range(first_true_idx, e_ins)) 
            idx_to_mutate = random.choice(candidates)

        # Selezione intelligente del vicino
        next_node = path[idx_to_mutate + 1][0]
        current_node = path[idx_to_mutate][0]
        
        # Nodi attivi nel segmento corrente (da escludere)
        active_in_segment = set(n for n, flag in path[s_ins:e_ins] if flag)
        
        # Filtro: esclude deposito, nodo corrente, e nodi già attivi nello stesso segmento
        neighbors = [n for n in graph.neighbors(next_node) 
                    if n != 0 and n != current_node and n not in active_in_segment]
        
        if not neighbors: 
            continue
        
        # Usa cache se disponibile, altrimenti calcola al volo
        if neighbor_distance_cache and next_node in neighbor_distance_cache:
            cached_dists = neighbor_distance_cache[next_node]
            scored_neighbors = []
            for neighbor in neighbors:
                if neighbor in cached_dists:
                    dist = cached_dists[neighbor]
                    collision_penalty = 1.0 if neighbor in active_nodes else 0.5
                    score = dist * collision_penalty
                    scored_neighbors.append((score, neighbor))
        else:
            # Fallback: calcolo al volo
            next_node_coords = graph.nodes[next_node]['pos']
            scored_neighbors = []
            
            for neighbor in neighbors:
                n_coords = graph.nodes[neighbor]['pos']
                dist = ((n_coords[0] - next_node_coords[0])**2 + 
                        (n_coords[1] - next_node_coords[1])**2) ** 0.5
                collision_penalty = 1.0 if neighbor in active_nodes else 0.5
                score = dist * collision_penalty
                scored_neighbors.append((score, neighbor))
        
        if not scored_neighbors:
            continue
        
        # HEAPQ PARTIAL SORT
        k = max(1, min(len(scored_neighbors) // 2, 4))
        best_k = heapq.nsmallest(k, scored_neighbors)
        
        # Weighted random
        scores_inv = [(1.0 / (s + 0.1), n) for s, n in best_k]
        total_weight = sum(w for w, _ in scores_inv)
        weights = [w / total_weight for w, _ in scores_inv]
        
        new_node = random.choices([n for _, n in scores_inv], weights=weights, k=1)[0]
        
        # Controllo duplicati
        if new_node in active_nodes:
            # Double mutation
            dup_index = next(i for i, (n, flag) in enumerate(path) 
                           if n == new_node and flag)
            s_dup, e_dup = founding_start_and_end_index(path, dup_index)
            
            return execute_double_segment_mutation(
                path, problem_instance, 
                (s_ins, e_ins, idx_to_mutate, new_node), 
                (s_dup, e_dup, new_node),
                graph
            )
        else:
            # Single mutation
            return execute_single_segment_mutation(
                path, problem_instance, s_ins, e_ins, idx_to_mutate, new_node, graph
            )

    return path, (0, 0, [])  # Fallito dopo max tentativi


# ---------------------------------------------------------------------------
# LOGICA DI RICOSTRUZIONE ROBUSTA
# ---------------------------------------------------------------------------

def execute_double_segment_mutation(full_path, problem, ins_data, rem_data, graph=None):
    if graph is None:
        graph = problem.graph
        
    s_ins, e_ins, idx_ins_abs, new_node = ins_data
    s_rem, e_rem, node_to_rem = rem_data
    
    # 1. Ordinamento segmenti
    if s_ins < s_rem:
        first_type = "ins"
        s1, e1 = s_ins, e_ins
        s2, e2 = s_rem, e_rem
        args1 = (idx_ins_abs - s1, new_node, graph)
        args2 = (node_to_rem,)
    else:
        first_type = "rem"
        s1, e1 = s_rem, e_rem
        s2, e2 = s_ins, e_ins
        args1 = (node_to_rem,)
        args2 = (idx_ins_abs - s2, new_node, graph)
        
    # 2. Estrazione e Modifica
    seg1 = full_path[s1 : e1+1]
    seg2 = full_path[s2 : e2+1]
    
    res_old1 = calculate_full_path_cost_final(problem, seg1)
    res_old2 = calculate_full_path_cost_final(problem, seg2)
    
    if first_type == "ins":
        seg1 = apply_insertion(seg1, *args1)
        seg2 = apply_removal(seg2, *args2)
    else:
        seg1 = apply_removal(seg1, *args1)
        seg2 = apply_insertion(seg2, *args2)
        
    res_new1 = calculate_full_path_cost_final(problem, seg1)
    res_new2 = calculate_full_path_cost_final(problem, seg2)
    
    # 3. Delta
    d_infeas = (res_new1[0] - res_old1[0]) + (res_new2[0] - res_old2[0])
    d_cost = (res_new1[1] - res_old1[1]) + (res_new2[1] - res_old2[1])
    inf_nodes = []
    if len(res_new1) > 2: inf_nodes.extend(res_new1[2])
    if len(res_new2) > 2: inf_nodes.extend(res_new2[2])
    
    delta = (d_infeas, d_cost, inf_nodes)
    
    # 4. RICOSTRUZIONE
    new_full_path = full_path[:s1] 
    new_full_path += seg1
    gap = full_path[e1+1 : s2]
    new_full_path += gap
    
    if len(gap) == 0:
        new_full_path += seg2[1:]
    else:
        new_full_path += seg2
        
    new_full_path += full_path[e2+1:]
    
    return new_full_path, delta


def execute_single_segment_mutation(full_path, problem, start, end, idx_abs, new_node, graph=None):
    if graph is None:
        graph = problem.graph
        
    segment = full_path[start : end + 1]
    idx_rel = idx_abs - start 
    old_res = calculate_full_path_cost_final(problem, segment) 
    
    segment_mutated = apply_insertion(segment, idx_rel, new_node, graph)
    
    new_res = calculate_full_path_cost_final(problem, segment_mutated)
    delta = (new_res[0] - old_res[0], new_res[1] - old_res[1], new_res[2] if len(new_res)>2 else [])
    
    new_full_path = full_path[:start] + segment_mutated + full_path[end+1:]
    return new_full_path, delta

def apply_insertion(segment, idx_rel, new_node, graph):
    current_node = segment[idx_rel][0]
    try:
        path_link = nx.shortest_path(graph, current_node, new_node, weight='dist')
        link_nodes = [(n, False) for n in path_link[1:-1]]
        return segment[:idx_rel+1] + link_nodes + [(new_node, True)] + segment[idx_rel+1:]
    except:
        return segment

def apply_removal(segment, node_to_remove):
    actives = [x for x in segment if x[1]]
    # Se il segmento collassa a un solo nodo attivo che stiamo rimuovendo,
    # restituiamo un "viaggio nullo" [0] che la logica di ricostruzione unirà.
    if len(actives) == 1 and actives[0][0] == node_to_remove:
        return [segment[0]] # Ritorna [0]
        
    new_seg = []
    for n, act in segment:
        if n == node_to_remove and act:
            new_seg.append((n, False))
        else:
            new_seg.append((n, act))
    return new_seg

In [18]:
def hill_climbing(problem_instance, initial_solution, n_iterations=1000):
    """
    Pure Hill Climbing (First Choice).
    Accetta solo mosse migliorative.
    """
    
    # 1. Inizializzazione
    current_solution = initial_solution.copy()
    
    # Calcolo costo iniziale COMPLETO una volta sola
    curr_infeas, curr_cost= calculate_full_path_cost_final(problem_instance, current_solution)
    
    # Pre-calcolo cache distanze E grafo (una volta sola)
    neighbor_distance_cache = precompute_neighbor_distances(problem_instance)
    path_list = neighborhood_greedy_strategy_dijistra(problem_instance)
    graph = problem_instance.graph  # Cache il grafo!
    
    # Loop di ottimizzazione
    for iteration in range(n_iterations):
        
        # 2. Genera un vicino e ottieni la DELTA
        neighbor_solution, delta_tuple = mutation_neighbor_of_next_insertion_only(
            current_solution, problem_instance, path_list, neighbor_distance_cache, graph
        )
        
        delta_infeas = delta_tuple[0]
        delta_cost = delta_tuple[1]
        
        # 3. Logica PURE HILL CLIMBING
        accept = False
        
        # Caso A: L'infeasibility diminuisce (MOLTO BENE) -> Accetta sempre
        if delta_infeas < 0:
            accept = True
            
        # Caso B: L'infeasibility aumenta (MALE) -> Rifiuta sempre
        elif delta_infeas > 0:
            accept = False
            
        # Caso C: L'infeasibility è invariata (uguale a prima)
        else: # delta_infeas == 0
            # Accetta solo se il costo diminuisce (o è uguale, per muoversi su plateau)
            if delta_cost <= 0:
                accept = True
            else:
                accept = False
        
        # 4. Aggiornamento (Solo se accettato)
        if accept:
            current_solution = neighbor_solution
            curr_infeas += delta_infeas
            curr_cost += delta_cost

    return current_solution, curr_cost

In [19]:
P=Problem(1000, density=0.2, alpha=1, beta=2)

In [20]:
y=neighborhood_greedy_strategy_dijistra(P)
y=choice_a_path(y,seed=random.randint(0,1000))
print(calculate_full_path_cost_final(P,y))

new_path, new_cost = hill_climbing(P,y, n_iterations=10)
print(new_path)
print(new_cost)
print(P.baseline())
print(calculate_full_path_cost_final(P,new_path))

(0, np.float64(46032459.235689834))
[(0, False), (941, False), (655, True), (941, True), (0, False), (935, False), (618, False), (115, True), (618, True), (935, True), (0, False), (151, False), (32, False), (26, True), (32, True), (151, True), (0, False), (766, True), (0, False), (286, True), (0, False), (411, False), (527, False), (255, True), (527, True), (411, True), (0, False), (233, True), (0, False), (185, False), (561, False), (146, True), (561, True), (185, True), (0, False), (40, False), (770, True), (40, True), (0, False), (237, False), (481, False), (108, True), (481, True), (237, True), (0, False), (199, False), (711, True), (199, True), (0, False), (902, False), (781, True), (902, True), (0, False), (151, False), (939, True), (151, False), (0, False), (802, False), (576, True), (802, True), (0, False), (874, False), (93, True), (874, True), (0, False), (402, False), (625, True), (402, True), (0, False), (607, False), (449, True), (607, True), (0, False), (235, False), (35,

In [21]:
calculate_full_path_cost_final(P,new_path)

(0, np.float64(45942965.710431375))

In [22]:
import random
import networkx as nx

def crossover_zero_paths_with_delta(parent1, parent2, possible_paths, problem_instance):
    """
    Esegue crossover e restituisce (nuovo_individuo, delta_tuple).
    delta_tuple = (delta_infeasibility, delta_cost, infeasible_nodes_list)
    """
    
    # Copia per non modificare gli originali fuori dalla funzione se necessario
    # (anche se qui lavoriamo costruendo new_individual, parent2 viene modificato nel processo)
    p2_working = parent2.copy()
    
    # --- 1. SELEZIONE SEGMENTO PARENT 1 ---
    start_index_1 = random.randint(1, len(parent1) - 2)
    start_index_1, end_index_1 = founding_start_and_end_index(parent1, start_index_1)
    
    segment_p1 = parent1[start_index_1 : end_index_1 + 1]
    
    # Identifica l'elemento GOLD pivot in Parent 1
    gold_element = 0
    index_gold_element_1 = -1
    for i in range(start_index_1, end_index_1):
        if parent1[i][1] == True:
            gold_element = parent1[i][0]
            index_gold_element_1 = i
            break
            
    # Se non troviamo gold element (segmento vuoto), abortiamo o gestiamo
    if gold_element == 0:
        return p2_working, (0, 0, [])

    list_gold_1 = find_True_flags(parent1[index_gold_element_1+1 : end_index_1]) # Nota: controlla se l'indice è corretto rispetto alla tua logica

    # --- 2. IDENTIFICAZIONE SEGMENTO PARENT 2 ---
    index_gold_element_2 = find_node_with_flag_true(p2_working, gold_element)
    
    try:
        start_index_2, end_index_2 = founding_start_and_end_index(p2_working, index_gold_element_2)
    except (IndexError, TypeError):
        # Fallback se la struttura non è coerente
        return p2_working, (0, 0, [])

    segment_p2_original = p2_working[start_index_2 : end_index_2 + 1]
    
    # Calcolo costi dei segmenti principali (SWAP)
    # Costo di ciò che entra (Parent 1 segment)
    res_seg1 = calculate_full_path_cost_final(problem_instance, segment_p1)
    # Costo di ciò che esce (Parent 2 segment)
    res_seg2 = calculate_full_path_cost_final(problem_instance, segment_p2_original)
    
    # Inizializza Delta con la differenza dello Swap
    delta_infeas = res_seg1[0] - res_seg2[0]
    delta_cost = res_seg1[1] - res_seg2[1]
    infeas_nodes = []
    if len(res_seg1) > 2: infeas_nodes.extend(res_seg1[2])

    # --- 3. GESTIONE SIDE EFFECTS (RIMOZIONE DUPLICATI ESTERNI) ---
    list_gold_2 = find_True_flags(p2_working[start_index_2 : end_index_2])
    
    # Qui accumuliamo la delta delle rimozioni esterne
    if len(list_gold_1) > 0:
        for node in list_gold_1:
            if node not in list_gold_2:
                # Questo nodo arriva da P1, ma non è nel segmento che stiamo togliendo da P2.
                # Se P2 ce l'ha altrove, dobbiamo rimuoverlo.
                
                # 1. Troviamo dove è ORA (prima di rimuoverlo) per calcolare il costo vecchio
                idx_external = find_node_with_flag_true(p2_working, node)
                
                # Se esiste (idx != -1) e non è nel segmento che stiamo già rimuovendo
                # (Il controllo 'not in list_gold_2' dovrebbe garantirlo, ma idx aiuta)
                if idx_external != -1:
                    s_ext, e_ext = founding_start_and_end_index(p2_working, idx_external)
                    seg_ext_old = p2_working[s_ext : e_ext + 1]
                    res_ext_old = calculate_full_path_cost_final(problem_instance, seg_ext_old)
                    
                    # 2. Eseguiamo la rimozione
                    # Nota: remove_more_gold_nodes aggiorna p2_working e gli indici del segmento principale
                    p2_working, start_index_2, end_index_2 = remove_more_gold_nodes(
                        p2_working, node, start_index_2, end_index_2
                    )
                    
                    # 3. Troviamo il segmento nuovo (dove era il nodo) per il costo nuovo
                    # Attenzione: gli indici sono cambiati. Usiamo founding intorno alla vecchia posizione approssimata 
                    # o cerchiamo il buco. Poiché remove_more_gold_nodes potrebbe aver collassato il segmento,
                    # dobbiamo gestire il caso.
                    # Semplificazione: se remove_more_gold_nodes mette solo False, il segmento è lì.
                    # Se rimuove nodi, è più complesso. Assumiamo la logica sicura:
                    
                    # Poiché non sappiamo dove sono finiti gli indici esatti del side-effect, 
                    # ricalcoliamo il costo sul "nuovo" segmento nella stessa posizione relativa se possibile,
                    # o accettiamo che remove_more_gold_nodes ritorni il "nuovo percorso parziale"?
                    # Per ora assumiamo che p2_working sia aggiornato.
                    
                    # Se il segmento è stato rimosso del tutto (es. conteneva solo quel nodo), 
                    # remove_more_gold_nodes dovrebbe aver gestito la fusione. 
                    # Calcolare la delta puntuale qui è rischioso senza funzioni helper dedicate.
                    
                    # APPROCCIO ROBUSTO PER DELTA SIDE-EFFECT:
                    # Invece di impazzire con gli indici, usiamo l'approccio "Segmento Isolato" 
                    # se hai implementato 'find_path_removed' o simili.
                    # Altrimenti, calcoliamo approssimativamente:
                    
                    # Cerchiamo il segmento modificato usando un indice vicino a idx_external (se valido)
                    # Se idx_external > len(p2_working), usiamo l'ultimo.
                    check_idx = min(idx_external, len(p2_working)-2)
                    s_ext_new, e_ext_new = founding_start_and_end_index(p2_working, check_idx)
                    seg_ext_new = p2_working[s_ext_new : e_ext_new + 1]
                    res_ext_new = calculate_full_path_cost_final(problem_instance, seg_ext_new)
                    
                    # Aggiorna Delta
                    delta_infeas += (res_ext_new[0] - res_ext_old[0])
                    delta_cost += (res_ext_new[1] - res_ext_old[1])
                    if len(res_ext_new) > 2: infeas_nodes.extend(res_ext_new[2])

    # --- 4. GESTIONE RECUPERO ORO PERSO (APPEND) ---
    # Ricalcoliamo list_gold_2 su p2_working aggiornato (anche se il segmento 2 non dovrebbe essere cambiato dai side effects esterni)
    # O usiamo la lista originale, dato che stiamo "sostituendo" quel blocco logico.
    # Usiamo la logica originale:
    
    if gold_element in list_gold_2:
        list_gold_2.remove(gold_element)
    
    list_2_not_in_1 = [node for node in list_gold_2 if node not in list_gold_1]

    new_path_append = []
    if len(list_2_not_in_1) > 0:
        new_path_append = insert_more_gold_nodes(list_2_not_in_1, possible_paths)
        
        # Calcolo Delta per l'append
        # new_path_append è un percorso completo 0->...->0.
        res_append = calculate_full_path_cost_final(problem_instance, new_path_append)
        
        delta_infeas += res_append[0]
        delta_cost += res_append[1]
        if len(res_append) > 2: infeas_nodes.extend(res_append[2])

    # --- 5. COSTRUZIONE INDIVIDUO ---
    # Attenzione: usiamo p2_working che ha subito le rimozioni dei side-effects
    # Segmento 1 viene da Parent 1 (che è intatto)
    # Segmento 2 viene rimosso da p2_working
    # Append viene aggiunto
    
    # Fix per evitare doppi zeri:
    # new_path_append[1:] rimuove lo 0 iniziale.
    # p2_working[end_index_2:] inizia con 0.
    
    new_individual = (
        p2_working[:start_index_2] + 
        segment_p1 +                 # Include start(0) e end(0)
        p2_working[end_index_2+1:] + # Salta lo 0 iniziale del resto di P2 (perché segment_p1 finisce con 0)
        new_path_append[1:]          # Salta lo 0 iniziale dell'append (perché p2 finisce con 0)
    )

    # Nota sulla costruzione:
    # segment_p1: 0...0
    # p2_working[:start_index_2]: ...X (finisce prima dello 0 di start) -> OK
    # p2_working[end_index_2+1:]: Y... (inizia dopo lo 0 di end) -> OK
    # new_path_append[1:]: A...0 (inizia dopo lo 0)
    
    # Verifica: p2_working[end_index_2:] include lo 0. Se segment_p1 finisce con 0, avremmo 0,0.
    # Quindi usiamo end_index_2 + 1 per tagliare lo 0 di rientro di p2.
    
    return new_individual, (delta_infeas, delta_cost, infeas_nodes)

In [23]:
def create_population(problem_instance, population_size, proportion=0.8):
    population = []
    near_list=neighborhood_greedy_strategy_dijistra(problem_instance=problem_instance)
    for n in range(population_size):
        # if random.random() < proportion:

        individual=choice_a_path(near_list, seed=n)
        # else:
        #     individual=full_path=choice_a_path(neighborhood_greedy_strategy_random(problem_instance=problem_instance, seed=n),seed=n+1)
        population.append(individual)
    return population

In [24]:
def tournament_selection(population_with_fitness, tournament_size=3):
    """
    Seleziona un genitore sfidando k individui a caso.
    
    Args:
        population_with_fitness: Lista di tuple [(path, (infeas, cost, ...)), ...]
        tournament_size (k): Quanti individui partecipano al torneo (default 3-5).
                             k più alto = più pressione selettiva (convergenza veloce).
                             k più basso = più diversità (esplorazione).
    """
    # 1. Prendi k candidati a caso dalla popolazione
    # Usa random.sample che è efficiente e senza rimpiazzo
    candidates = random.sample(population_with_fitness, tournament_size)
    
    # 2. Trova il vincitore (quello con la fitness minore)
    # La fitness è una tupla (infeasibility, cost, ...).
    # Python confronta: prima infeasibility, se pari confronta cost.
    # key=lambda x: x[1] dice di guardare solo la parte fitness per il confronto.
    
    winner = min(candidates, key=lambda x: (x[1][0], x[1][1]))
    
    # 3. Restituisci il genoma (il percorso) e la sua fitness (utile per delta dopo)
    return winner[0], winner[1]

In [25]:
import random

def precompute_neighbor_distances(problem_instance):
    """
    Pre-calcola tutte le distanze euclidee tra nodi e loro vicini (una volta sola).
    Ritorna un cache O(1) per le operazioni di mutazione.
    """
    graph = problem_instance.graph
    cache = {}
    
    for node in graph.nodes():
        neighbors_with_dist = {}
        node_coords = graph.nodes[node]['pos']
        
        for neighbor in graph.neighbors(node):
            if neighbor != 0:  # Esclude deposito
                n_coords = graph.nodes[neighbor]['pos']
                dist = ((n_coords[0] - node_coords[0])**2 + 
                        (n_coords[1] - node_coords[1])**2) ** 0.5
                neighbors_with_dist[neighbor] = dist
        
        cache[node] = neighbors_with_dist
    
    return cache

import random

def genetic_algorithm(problem_instance, population_size=100, n_generations=50, temperature=0.8):
    """
    temperature (0.0 - 1.0): Determina quanto aggressivamente sale la probabilità di mutazione.
    STRATEGIA: Family Competition (Implicit Niching).
    Il figlio compete solo contro il genitore strutturalmente più simile (Parent2 per Crossover, Parent1 per Mutazione).
    
    tournament_k è dinamico: inizia da 2 e aumenta fino a 6 nel corso delle generazioni.
    """
    
    # 0. PRE-CALCOLO CACHE
    neighbor_distance_cache = precompute_neighbor_distances(problem_instance)
    graph = problem_instance.graph 
    
    # 1. Inizializzazione
    raw_population = create_population(problem_instance, population_size)
    population_data = []
    
    for ind in raw_population:
        fit = calculate_full_path_cost_final(problem_instance, ind)
        population_data.append((ind, fit))
        
    path_list = neighborhood_greedy_strategy_dijistra(problem_instance)

    # Helper per confrontare due fitness (Infeasibility prima, poi Costo)
    def is_better(fit_a, fit_b):
        # fit = (infeasible_count, total_cost, ...)
        if fit_a[0] != fit_b[0]:
            return fit_a[0] < fit_b[0] # Minore infeasibility è meglio
        return fit_a[1] < fit_b[1]     # Minore costo è meglio

    for generation in range(n_generations):
        new_population_data = []
        
        # --- GESTIONE TEMPERATURA ---
        progress = generation / n_generations
        
        # tournament_k dinamico: da 2 a 6
        min_tournament_k = 2
        max_tournament_k = 6
        current_tournament_k = int(min_tournament_k + progress * (max_tournament_k - min_tournament_k))
        current_tournament_k = max(2, min(6, current_tournament_k))  # Clamp tra 2 e 6
        
        # Probabilità di fare CROSSOVER (aumenta col tempo)
        base_crossover_rate = 0.1
        max_added_rate = 0.8
        current_crossover_prob = base_crossover_rate + (progress * temperature * max_added_rate)
        current_crossover_prob = max(0.0, min(1.0, current_crossover_prob))

        # Elitismo: Il migliore assoluto passa sempre gratis (assicurazione sulla vita)
        best_of_gen = min(population_data, key=lambda x: (x[1][0], x[1][1]))
        new_population_data.append(best_of_gen)
        
        while len(new_population_data) < population_size:
            # --- SELEZIONE ---
            parent1_path, p1_fit = tournament_selection(population_data, current_tournament_k)
            parent2_path, p2_fit = tournament_selection(population_data, current_tournament_k)
            
            # --- LOGICA COMPETITIVA ---
            winner_path = None
            winner_fit = None
            
           
            if random.random() > current_crossover_prob:
                # === MUTAZIONE ===
                # Generiamo il figlio da Parent 1
                child_path, mut_delta = mutation_neighbor_of_next_insertion_only(
                    parent1_path, problem_instance, path_list, neighbor_distance_cache, graph
                )
                
                m_infeas = p1_fit[0] + mut_delta[0]
                m_cost = p1_fit[1] + mut_delta[1]
                child_fit = (m_infeas, m_cost, [])

                # SURVIVOR SELECTION: Child vs Parent 1
                if is_better(child_fit, p1_fit):
                    winner_path, winner_fit = child_path, child_fit
                else:
                    winner_path, winner_fit = parent1_path, p1_fit

            else:
                # === CROSSOVER ===
                # Generiamo il figlio usando Parent 2 come base per il Delta
                child_path, delta_tuple = crossover_zero_paths_with_delta(
                    parent1_path, parent2_path, path_list, problem_instance
                )
                
                child_infeas = p2_fit[0] + delta_tuple[0]
                child_cost = p2_fit[1] + delta_tuple[1]
                child_fit = (child_infeas, child_cost, []) 
                
                # SURVIVOR SELECTION: Child vs Parent 2 (perché strutturalmente deriva da P2)
                if is_better(child_fit, p2_fit):
                    winner_path, winner_fit = child_path, child_fit
                else:
                    winner_path, winner_fit = parent2_path, p2_fit
            
            
            new_population_data.append((winner_path, winner_fit))
        
        # Aggiorna popolazione
        population_data = new_population_data
        
        # Monitoraggio
        if generation % 10 == 0 or generation == n_generations - 1:
            best_now = min(population_data, key=lambda x: (x[1][0], x[1][1]))
            print(f"Gen {generation} | K={current_tournament_k} | CrossoverRate: {current_crossover_prob:.2f} | Best: {best_now[1][1]:.2f}")

    return min(population_data, key=lambda x: (x[1][0], x[1][1]))

In [26]:
P=Problem(100, density=0.2, alpha=1, beta=2)

In [27]:
x=genetic_algorithm(P, population_size=70, n_generations=100)
print(x)
print(calculate_full_path_cost_final(P,x[0]))
print(P.baseline())

Gen 0 | K=2 | CrossoverRate: 0.10 | Best: 6115666.00
Gen 10 | K=2 | CrossoverRate: 0.16 | Best: 5736922.99
Gen 20 | K=2 | CrossoverRate: 0.23 | Best: 5509047.78
Gen 30 | K=3 | CrossoverRate: 0.29 | Best: 5337343.18
Gen 40 | K=3 | CrossoverRate: 0.36 | Best: 5210900.46
Gen 50 | K=4 | CrossoverRate: 0.42 | Best: 5133449.94
Gen 60 | K=4 | CrossoverRate: 0.48 | Best: 5085518.68
Gen 70 | K=4 | CrossoverRate: 0.55 | Best: 5055105.17
Gen 80 | K=5 | CrossoverRate: 0.61 | Best: 5034497.57
Gen 90 | K=5 | CrossoverRate: 0.68 | Best: 4990547.20
Gen 99 | K=5 | CrossoverRate: 0.73 | Best: 4965418.52
([(0, False), (51, True), (0, False), (77, True), (0, False), (53, False), (72, False), (24, True), (72, False), (53, False), (0, False), (53, False), (72, False), (25, False), (29, True), (25, True), (72, False), (53, False), (0, False), (1, False), (22, False), (1, True), (0, False), (53, False), (33, False), (66, False), (30, True), (66, True), (33, False), (53, False), (0, False), (1, False), (26, Tr

In [28]:
def mutate_trip_counts(trip_counts):
    """
    Copia trip_counts e aggiunge un valore random [1, 4] a un singolo elemento.
    Restituisce: (new_trip_counts, changed_index)
    """
    new_counts = trip_counts[:]  # Copia per non modificare l'originale
    
    # Sceglie un indice a caso (corrispondente a un circuito)
    idx_to_change = random.randint(0, len(new_counts) - 1)
    
    # Aggiunge valore tra 1 e 4
    increment = 2
    new_counts[idx_to_change] *= increment
    
    return new_counts, idx_to_change

In [29]:
def evaluate_trip_mutation_smart(problem_instance, path, old_counts, new_counts, changed_idx):
    """
    Calcola la delta riutilizzando calculate_full_path_cost_final 
    ma applicandola solo al segmento modificato.
    """
    
    # 1. Recupera i confini dei circuiti (come prima)
    boundaries = get_trip_boundaries(path)
    start_idx, end_idx = boundaries[changed_idx]
    
    # 2. Isola il "mini-path" (Slice)
    # Esempio: [(0, True), (A, True), (B, False), (0, True)]
    mini_path = path[start_idx : end_idx + 1]
    
    # 3. Prepara i trip_counts per questo singolo segmento (liste di 1 elemento)
    old_trip_count_list = [old_counts[changed_idx]]
    new_trip_count_list = [new_counts[changed_idx]]
    
    # 4. CHIAMA LA TUA FUNZIONE ORIGINALE sul mini-path
    # Nota: la funzione restituisce (infeasible_count, cost, ...)
    _, cost_old = calculate_full_path_cost_final(problem_instance, mini_path, old_trip_count_list)
    _, cost_new = calculate_full_path_cost_final(problem_instance, mini_path, new_trip_count_list)
    
    # 5. Calcola Delta
    delta = cost_new - cost_old
    
    return delta

In [30]:
def run_hill_climbing_trips(problem_instance, path, initial_trip_counts, max_iter=1000):
    """
    Esegue Hill Climbing per ottimizzare SOLO il vettore trip_counts,
    mantenendo fisso il percorso (path).
    """
    
    # 1. Inizializzazione
    current_counts = list(initial_trip_counts)
    
    # Calcolo costo iniziale completo (una tantum all'inizio)
    # Serve per avere il punto di partenza e per stampare il risultato finale corretto
    _, current_total_cost = calculate_full_path_cost_final(problem_instance, path, current_counts)
    
    
    iteration = 0
    improvements = 0
    
    while iteration < max_iter:
        iteration += 1
        
        # 2. Mutazione (proposta vicino)
        new_counts, changed_idx = mutate_trip_counts(current_counts)
        
        # 3. Valutazione Delta (Efficiente)
        delta = evaluate_trip_mutation_smart(
            problem_instance, 
            path, 
            current_counts, 
            new_counts, 
            changed_idx
        )
        
        # 4. Decisione (Hill Climbing: accetta solo se migliora)
        if delta < 0:
            # Trovato miglioramento!
            current_counts = new_counts
            current_total_cost += delta  # Aggiorna il costo totale sommando la delta
            improvements += 1
            
            
        else:
            # Nessun miglioramento, scarta new_counts (non facciamo nulla)
            pass

    print(f"\n--- Fine Ottimizzazione ---")
    print(f"Costo Finale: {current_total_cost:.4f}")
    print(f"Miglioramenti trovati: {improvements}")
    print(f"Counts Finali: {current_counts}")
    
    return current_counts, current_total_cost

In [31]:
def conversion_solution(problem_instance,full_path, trip_counts):
    """
    Converte il full_path in una lista dove nella tupla al posto di True/False, c'è la quantità di oro recuperata.
    A seconda del valore di trip count nel mio percorso devo fare n giri portando 1/n oro.
    Esempio: [(0, False), (A, True), (B, False), (C, True), (0, False)] 
    diventa [(0, 0), (A, 10), (B, 0), (C, 5), (0, 0)]   
    """
    converted_path = []
    list_of_bounds=get_trip_boundaries(full_path)
    for trip_idx in range(len(trip_counts)):
        start_idx, end_idx = list_of_bounds[trip_idx]
        trip_count = trip_counts[trip_idx]
        for num_trips in range(trip_count):
            for i in range(start_idx, end_idx):
                node, is_active = full_path[i]
                if is_active:
                    gold_dict = problem_instance.gold_dict 
                    total_gold = gold_dict[node]
                    gold_amount = total_gold / trip_count if trip_count > 0 else 0
                    converted_path.append((node, gold_amount))
                else:
                    converted_path.append((node, 0))

    converted_path.append((0, 0))  # Aggiungi il deposito finale
    converted_path=converted_path[1:]  # Rimuovi il deposito iniziale duplicato
    return converted_path


In [32]:
P=Problem(10, density=0.2, alpha=1, beta=2)
x=genetic_algorithm(P, population_size=70, n_generations=100)
print(x)
count_trips=[1 for _ in range(len(get_trip_boundaries(x[0])))]  
count_trips_2=[2 for _ in range(len(get_trip_boundaries(x[0])))] 
print(conversion_solution(P,x[0],count_trips))
print(conversion_solution(P,x[0],count_trips_2))

Gen 0 | K=2 | CrossoverRate: 0.10 | Best: 1838542.69
Gen 10 | K=2 | CrossoverRate: 0.16 | Best: 1763331.61
Gen 20 | K=2 | CrossoverRate: 0.23 | Best: 1763331.61
Gen 30 | K=3 | CrossoverRate: 0.29 | Best: 1763331.61
Gen 40 | K=3 | CrossoverRate: 0.36 | Best: 1763331.61
Gen 50 | K=4 | CrossoverRate: 0.42 | Best: 1763331.61
Gen 60 | K=4 | CrossoverRate: 0.48 | Best: 1763331.61
Gen 70 | K=4 | CrossoverRate: 0.55 | Best: 1763331.61
Gen 80 | K=5 | CrossoverRate: 0.61 | Best: 1763331.61
Gen 90 | K=5 | CrossoverRate: 0.68 | Best: 1763331.61
Gen 99 | K=5 | CrossoverRate: 0.73 | Best: 1763331.61
([(0, False), (7, False), (6, True), (7, False), (0, False), (1, True), (0, False), (7, False), (6, False), (5, True), (6, False), (7, False), (0, False), (8, False), (9, True), (8, True), (7, False), (0, False), (8, False), (9, False), (3, True), (9, False), (8, False), (7, False), (0, False), (7, False), (6, False), (5, False), (4, True), (5, False), (6, False), (7, False), (0, False), (7, False), (2, 

In [33]:
def solving_problem_with_ga_and_trip_optimization(problem_instance, population_size=100, n_generations=50, n_hill_climb_iterations=500):
    """
    Risolve il problema combinando Genetic Algorithm per il percorso
    e Hill Climbing per l'ottimizzazione dei trip counts.
    """
    
    # 1. Esegui il Genetic Algorithm per ottenere un buon percorso
    ga_result = genetic_algorithm(problem_instance, population_size, n_generations)
    best_path = ga_result[0]
    
    # 2. Inizializza i trip counts (1 viaggio per circuito)
    initial_trip_counts = [1 for _ in range(len(get_trip_boundaries(best_path)))]
    
    # 3. Esegui Hill Climbing per ottimizzare i trip counts
    optimized_trip_counts, final_cost = run_hill_climbing_trips(
        problem_instance, 
        best_path, 
        initial_trip_counts, 
        max_iter=n_hill_climb_iterations
    )
    
    return best_path, optimized_trip_counts, final_cost

In [ ]:
import time
import statistics

def benchmark_approach(approach_name, approach_func, n_repetitions=3, verbose=True):
    """
    Benchmarka un approccio, misurando tempo e costo finale.
    
    Args:
        approach_name: Nome dell'approccio (per i log)
        approach_func: Callable che esegue l'approccio e restituisce (path, cost)
        n_repetitions: Numero di volte da ripetere
        verbose: Se True, stampa i progressi
        
    Returns:
        dict con statistiche: {
            'times': lista dei tempi,
            'costs': lista dei costi finali,
            'mean_time': tempo medio,
            'std_time': deviazione std tempo,
            'mean_cost': costo medio,
            'std_cost': deviazione std costo,
            'min_cost': miglior costo trovato,
            'max_cost': peggior costo trovato
        }
    """
    times = []
    costs = []
    
    for rep in range(n_repetitions):
        if verbose:
            print(f"  {approach_name} - Ripetizione {rep + 1}/{n_repetitions}...", end=" ", flush=True)
        
        start = time.time()
        path, cost = approach_func()
        elapsed = time.time() - start
        
        times.append(elapsed)
        costs.append(cost)
        
        if verbose:
            print(f"Time: {elapsed:.2f}s, Cost: {cost:.2f}")
    
    # Calcolo statistiche
    result = {
        'times': times,
        'costs': costs,
        'mean_time': statistics.mean(times),
        'std_time': statistics.stdev(times) if len(times) > 1 else 0,
        'mean_cost': statistics.mean(costs),
        'std_cost': statistics.stdev(costs) if len(costs) > 1 else 0,
        'min_cost': min(costs),
        'max_cost': max(costs),
        'n_repetitions': n_repetitions
    }
    
    return result


def print_benchmark_results(results_dict, baseline_cost):
    """
    Stampa i risultati in una tabella formattata.
    
    Args:
        results_dict: Dict come {approach_name: benchmark_result, ...}
        baseline_cost: Costo baseline per il confronto
    """
    print("\n" + "="*100)
    print(f"{'Approccio':<20} {'Ripet':<6} {'Tempo (s)':<15} {'Costo':<15} {'vs Baseline':<15} {'Note':<20}")
    print("="*100)
    
    for name, result in results_dict.items():
        time_str = f"{result['mean_time']:.3f}±{result['std_time']:.3f}"
        cost_str = f"{result['mean_cost']:.2f}±{result['std_cost']:.2f}"
        vs_baseline = ((result['mean_cost'] - baseline_cost) / baseline_cost * 100)
        vs_str = f"{vs_baseline:+.1f}%"
        
        print(f"{name:<20} {result['n_repetitions']:<6} {time_str:<15} {cost_str:<15} {vs_str:<15}")
    
    print("="*100)


# ============================================================================
# SETUP BENCHMARK
# ============================================================================

# ============================================================================
# SETUP BENCHMARK
# ============================================================================

print("Creazione del problema (1000 nodi, density=0.2)...")
P = Problem(1000, density=1, alpha=1, beta=1)
baseline = P.baseline()
print(f"Baseline: {baseline:.2f}\n")

# Prepara i dati comuni (lista percorsi per gli algoritmi)
path_list = neighborhood_greedy_strategy_dijistra(P)

# ============================================================================
# APPROCCIO 1: GA + TRIP OPTIMIZATION STANDARD
# ============================================================================
def approach_ga_standard():
    """GA + Trip Optimization - Standard"""
    path, trip_counts, cost = solving_problem_with_ga_and_trip_optimization(
        P, population_size=15, n_generations=20, n_hill_climb_iterations=500
    )
    return path, cost

# ============================================================================
# APPROCCIO 2: GA + TRIP OPTIMIZATION (Aggressivo)
# ============================================================================
def approach_ga_hc():
    """GA + Trip Optimization - Aggressivo"""
    path, trip_counts, cost = solving_problem_with_ga_and_trip_optimization(
        P, population_size=25, n_generations=20, n_hill_climb_iterations=1000
    )
    return path, cost

# ============================================================================
# APPROCCIO 3: GA + TRIP OPTIMIZATION (Veloce)
# ============================================================================
def approach_ga_short():
    """GA + Trip Optimization - Veloce"""
    path, trip_counts, cost = solving_problem_with_ga_and_trip_optimization(
        P, population_size=15, n_generations=30, n_hill_climb_iterations=300
    )
    return path, cost

# ============================================================================
# APPROCCIO 4: GA + TRIP OPTIMIZATION (Equilibrato)
# ============================================================================
def approach_ga_medium():
    """GA + Trip Optimization - Equilibrato"""
    path, trip_counts, cost = solving_problem_with_ga_and_trip_optimization(
        P, population_size=30, n_generations=15, n_hill_climb_iterations=600
    )
    return path, cost

# ============================================================================
# APPROCCIO 5: GA + TRIP OPTIMIZATION (Leggero)
# ============================================================================
def approach_hc_standard():
    """GA + Trip Optimization - Leggero"""
    path, trip_counts, cost = solving_problem_with_ga_and_trip_optimization(
        P, population_size=10, n_generations=5, n_hill_climb_iterations=200
    )
    return path, cost


# ============================================================================
# STAMPA RISULTATI
# ============================================================================

print("AVVIO BENCHMARK (1 ripetizione per approccio)...\n")

benchmark_results = {}

print("1. GA Standard:")
benchmark_results['GA-Std'] = benchmark_approach('GA-Std', approach_ga_standard, n_repetitions=1)

print("\n2. GA Aggressivo:")
benchmark_results['GA-HC'] = benchmark_approach('GA-HC', approach_ga_hc, n_repetitions=1)

print("\n3. GA Veloce:")
benchmark_results['GA-Short'] = benchmark_approach('GA-Short', approach_ga_short, n_repetitions=1)

print("\n4. GA Equilibrato:")
benchmark_results['GA-Medium'] = benchmark_approach('GA-Medium', approach_ga_medium, n_repetitions=1)

print("\n5. GA Leggero:")
benchmark_results['HC-Std'] = benchmark_approach('HC-Std', approach_hc_standard, n_repetitions=1)

# ============================================================================
# STAMPA RISULTATI
# ============================================================================

print_benchmark_results(benchmark_results, baseline)

Creazione del problema (1000 nodi, density=0.2)...
Baseline: 192936.23

AVVIO BENCHMARK (1 ripetizione per approccio)...

1. GA Standard:
  GA-Std - Ripetizione 1/1... Gen 0 | K=2 | CrossoverRate: 0.10 | Best: 192936.21
Gen 10 | K=4 | CrossoverRate: 0.42 | Best: 192936.20


In [ ]:

def mutation_neighbor_of_next_insertion_only(path, problem_instance, path_list, neighbor_distance_cache=None, graph=None):
    if graph is None:
        graph = problem_instance.graph
    
    active_nodes = set(n for n, flag in path if flag)
    active_count = len(active_nodes)
    total_nodes = len(graph)
    occupancy = active_count / total_nodes
    
    max_retries = max(1, 4 - int(occupancy * 3))
    
    for attempt in range(max_retries):
        random_index = random.randint(1, len(path) - 2)
        s_ins, e_ins = founding_start_and_end_index(path, random_index)
        
        first_true_idx = -1
        for i in range(s_ins, e_ins):
            if path[i][1]:
                first_true_idx = i
                break
        
        if first_true_idx == -1: 
            continue 

        if first_true_idx + 1 >= e_ins:
            idx_to_mutate = first_true_idx
        else:
            candidates = list(range(first_true_idx, e_ins)) 
            idx_to_mutate = random.choice(candidates)

        next_node = path[idx_to_mutate + 1][0]
        current_node = path[idx_to_mutate][0]
        
        active_in_segment = set(n for n, flag in path[s_ins:e_ins] if flag)
        
        # Se ho la cache, prendo i 5 più vicini da lì
        if neighbor_distance_cache and next_node in neighbor_distance_cache:
            cached_neighbors = neighbor_distance_cache[next_node]
            # Filtra escludendo: il nodo successivo, il nodo corrente, quelli già attivi nel segmento
            candidates = [n for n in cached_neighbors.keys() 
                         if n != 0 and n != current_node and n != next_node and n not in active_in_segment]
            
            if not candidates:
                continue
            
            # Sceglie uno casuale fra i candidati
            new_node = random.choice(candidates)
        else:
            # Fallback: usa il metodo originale se non ha cache
            neighbors = [n for n in graph.neighbors(next_node) 
                        if n != 0 and n != current_node and n not in active_in_segment]
            
            if not neighbors: 
                continue
            
            new_node = random.choice(neighbors)
        
        if new_node in active_nodes:
            dup_index = next(i for i, (n, flag) in enumerate(path) 
                           if n == new_node and flag)
            s_dup, e_dup = founding_start_and_end_index(path, dup_index)
            
            return execute_double_segment_mutation(
                path, problem_instance, 
                (s_ins, e_ins, idx_to_mutate, new_node), 
                (s_dup, e_dup, new_node),
                graph
            )
        else:
            return execute_single_segment_mutation(
                path, problem_instance, s_ins, e_ins, idx_to_mutate, new_node, graph
            )

    return path, (0, 0, [])


def execute_double_segment_mutation(full_path, problem, ins_data, rem_data, graph=None):
    if graph is None:
        graph = problem.graph
        
    s_ins, e_ins, idx_ins_abs, new_node = ins_data
    s_rem, e_rem, node_to_rem = rem_data
    
    if s_ins < s_rem:
        first_type = "ins"
        s1, e1 = s_ins, e_ins
        s2, e2 = s_rem, e_rem
        args1 = (idx_ins_abs - s1, new_node, graph)
        args2 = (node_to_rem,)
    else:
        first_type = "rem"
        s1, e1 = s_rem, e_rem
        s2, e2 = s_ins, e_ins
        args1 = (node_to_rem,)
        args2 = (idx_ins_abs - s2, new_node, graph)
        
    seg1 = full_path[s1 : e1+1]
    seg2 = full_path[s2 : e2+1]
    
    res_old1 = calculate_full_path_cost_final(problem, seg1)
    res_old2 = calculate_full_path_cost_final(problem, seg2)
    
    if first_type == "ins":
        seg1 = apply_insertion(seg1, *args1)
        seg2 = apply_removal(seg2, *args2)
    else:
        seg1 = apply_removal(seg1, *args1)
        seg2 = apply_insertion(seg2, *args2)
        
    res_new1 = calculate_full_path_cost_final(problem, seg1)
    res_new2 = calculate_full_path_cost_final(problem, seg2)
    
    d_infeas = (res_new1[0] - res_old1[0]) + (res_new2[0] - res_old2[0])
    d_cost = (res_new1[1] - res_old1[1]) + (res_new2[1] - res_old2[1])
    inf_nodes = []
    if len(res_new1) > 2: inf_nodes.extend(res_new1[2])
    if len(res_new2) > 2: inf_nodes.extend(res_new2[2])
    
    delta = (d_infeas, d_cost, inf_nodes)
    
    new_full_path = full_path[:s1] 
    new_full_path += seg1
    gap = full_path[e1+1 : s2]
    new_full_path += gap
    
    if len(gap) == 0:
        new_full_path += seg2[1:]
    else:
        new_full_path += seg2
        
    new_full_path += full_path[e2+1:]
    
    return new_full_path, delta


def execute_single_segment_mutation(full_path, problem, start, end, idx_abs, new_node, graph=None):
    if graph is None:
        graph = problem.graph
        
    segment = full_path[start : end + 1]
    idx_rel = idx_abs - start 
    old_res = calculate_full_path_cost_final(problem, segment) 
    
    segment_mutated = apply_insertion(segment, idx_rel, new_node, graph)
    
    new_res = calculate_full_path_cost_final(problem, segment_mutated)
    delta = (new_res[0] - old_res[0], new_res[1] - old_res[1], new_res[2] if len(new_res)>2 else [])
    
    new_full_path = full_path[:start] + segment_mutated + full_path[end+1:]
    return new_full_path, delta

def apply_insertion(segment, idx_rel, new_node, graph):
    current_node = segment[idx_rel][0]
    try:
        path_link = nx.shortest_path(graph, current_node, new_node, weight='dist')
        link_nodes = [(n, False) for n in path_link[1:-1]]
        return segment[:idx_rel+1] + link_nodes + [(new_node, True)] + segment[idx_rel+1:]
    except:
        return segment

def apply_removal(segment, node_to_remove):
    actives = [x for x in segment if x[1]]
    if len(actives) == 1 and actives[0][0] == node_to_remove:
        return [segment[0]]
        
    new_seg = []
    for n, act in segment:
        if n == node_to_remove and act:
            new_seg.append((n, False))
        else:
            new_seg.append((n, act))
    return new_seg